<a href="https://colab.research.google.com/github/lee1201zxc/study/blob/main/AI/ML/05_2_Validate_GridSearch_RandomSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 교차 검증, 그리드 서치
테스트 세트는 가장 마지막에 한 번만 사용하는 것이 좋음, 그렇지 않으면 과적합, 그래서 검증 세트로 모델을 평가,<br><br>

과적합 이유 : score는 모델 파라미터를 바꾸지 않으나 개발자(사람이)가 수치를 보고 테스트 셋의 score가 올라가도록 파라미터를 설정함 -> 모델에 테스트 셋 결과가 들어감. 그래서 중간엔 validation(평가)세트로 파라미터 수정하고 마지막에 테스트 셋을 이용함
<br><br>
검증 세트 : 하이퍼 파라미터 선택, 모델 비교
<br><br>
보통 훈련, 검증, 테스트 비율을 6:2:2정도로 함(훈련 데이터 많으면 더 줄이고)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
# 훈련 데이터를 다시 훈련 데이터 , 검증 데이터로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
#훈련 세특에 오버피팅
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077


## K폴드 교차 검증
훈련 데이터를 k개나누어 k-1은 훈련 데이터 ,1개는 검증 데이터, 이걸 k번 나눈다음 각각 점수를 계산하고 평균냄
기본적으로 데이터 섞지 않음
<br><br>
존재 이유 : 모델 평가의 운과 불안정성 줄임

In [4]:
from sklearn.model_selection import cross_validate

#기본적으로 5-폴드 교차 검증
# fit_time -> 각 세트별 훈련 시간, score_time -> 검증 시간, test_score -> 최종 점수
scores= cross_validate(dt, train_input, train_target)
print(scores)
# test_score평균 값 -> 교차 검증의 최종 점수
print(np.mean(scores['test_score']))


#데이터를 클래스 비율대로 나누도록함(치우치지 않도록), 기본적으로 cross_validate엔 StratifiedKFold있음
from sklearn.model_selection import StratifiedKFold
#n_splits-> 분할 개수, shuffle=True -> 섞기
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores= cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))



{'fit_time': array([0.0111227 , 0.01002479, 0.01110697, 0.01010728, 0.01073885]), 'score_time': array([0.00252771, 0.00243592, 0.002635  , 0.00242257, 0.00252485]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719


## 하이퍼 파라미터 튜닝

모델이 학습하는 것이 아닌 사람이 직접 지정해줘야하는 파라미터

###그리드 서치

사이킷런에서 제공하는 클래스, 하이퍼파라미터 탐색, 교차 검증을 한 번에 수행

In [15]:
from sklearn.model_selection import GridSearchCV

# 검증을 진행할 파라미터 값 나열
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# 그리드 서치 객체 생성(모든 모델 가), params로 파라미터 값 전달
# n_jobs=-1 -> 병렬 실행에 사용할 코어 수 지정, -1 -> 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# 교차 검증 점수가 가장 높은 모델 가져옴
dt=gs.best_estimator_
print("가장 점수 높은 모델의 점수")
print(dt.score(train_input, train_target))
print()

#가장 점수 높은 파라미터 값
print(gs.best_params_)

# 각 교차 검증으로 얻은 점수
print()
print("각 교차 검증으로 얻은 점수")
print(gs.cv_results_['mean_test_score'])

#best_index_를 통해 가장 점수 높은 파라미터 값의 인덱스 값 얻음
print(gs.cv_results_['params'][gs.best_index_])

print("-----------------------")

# 더욱더 복잡한 예제, 여러 파라미터 바꾸기
params ={
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5,20,1),
    'min_samples_split' : range(2,100,10)

}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

가장 점수 높은 모델의 점수
0.9615162593804117

{'min_impurity_decrease': 0.0001}

각 교차 검증으로 얻은 점수
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
-----------------------
{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


### 랜덤 서치
모든 조합을 다 보지 않고 무작위로 일부 조합만 샘플링함

빠름

In [24]:
from scipy.stats import uniform, randint
# uniform -> 실수값, radient -> 정수값을 주어진 범위 내에서 "고르게" 뽑음

rgen= randint(0,10)
print(rgen.rvs(5))

ugen= uniform(0,10)
print(ugen.rvs(10))

print("-----------------------")

#이 사이의 값을 샘플링함
params ={
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_samples_leaf' : randint(1,25)

}

from sklearn.model_selection import RandomizedSearchCV
rs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)
# 최적 파라미터
print(rs.best_params_)
print(np.max(rs.cv_results_['mean_test_score']))
dt=rs.best_estimator_
print(dt.score(test_input, test_target))


[5 3 4 9 5]
[2.7393178  0.36686099 1.00059291 3.89593892 4.04997313 7.83967841
 4.79922581 5.5189218  1.2738661  9.68499107]
-----------------------
{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
